In [1]:
import pandas as pd
import re
import os
import operator
from Bio.Seq import Seq
from fuzzysearch import find_near_matches
from difflib import SequenceMatcher
from Bio import pairwise2
import csv

def complement(sequence):
    sequence = sequence.upper()
    sequence = sequence.replace('A', 't')
    sequence = sequence.replace('T', 'a')
    sequence = sequence.replace('C', 'g')
    sequence = sequence.replace('G', 'c')
    return sequence.upper()

def reverse(sequence):
    sequence = sequence.upper()
    return sequence[::-1]

def complement_reverse(sequence):
    return complement(reverse(sequence))

def mkdir(path):
    path=path.strip()
    path=path.rstrip("\\")
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path) 


def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [ ]:
import tkinter as tk
from tkinter import messagebox
import threading

# 定义一个全局变量，用于标记是否停止运行
stop_running = False



# 定义一个函数，用于运行你的代码模块
def module1(bar_csv,com_fq):

    global df
    df = pd.read_csv(bar_csv, index_col=None, header=0,sep = ',',encoding="utf-8")
    n_list = []#存初始名字
    for i in range(df.shape[0]):#range(df.shape[0])
    #     p=0
        rf = open("./{}".format(com_fq))
        name = df['name'][i]
        n_list.append(name)
        name_2 = name[0:name.rfind('-')]
        mkdir('./indel数据/{}'.format(name_2))
        barcode = df['barcode'][i]
        re_barcode = complement_reverse(barcode)
        print (name,barcode,re_barcode)
        A1 = open("./indel数据/{}/{}.txt".format(name_2,name),'w')
        for line in rf:
    #        p+=1
            line = line.strip()
            if ( barcode in line):
                A1.write(line)
                A1.write('\n')
            elif ( re_barcode in line):
                A1.write(complement_reverse(line))
                A1.write('\n')
        A1.close()
        A2 = open("./indel数据/{}/{}.txt".format(name_2,name))
        count_dict = {}
        for line2 in A2:
            line2 = line2.rstrip()#消除/n
            count = count_dict.setdefault(line2, 0)
            count += 1
            count_dict[line2] = count
        sorted_count_dict = sorted(count_dict.items(), key=operator.itemgetter(1), reverse=True)
        A3 = open("./indel数据/{}/{}.txt".format(name_2,name),'w')
        A3.write('Grouping sequence'+'\t'+'\t'+'\t'+'Count')
        A3.write('\n')
        for item in sorted_count_dict:
            A3.write(item[0]+'\t'+str(item[1]))
            A3.write('\n')
        print(i+1)
        A2.close()
        A3.close()
        rf.close()
    #module2(file_name,lr_txt,mediant,df)
        
        
###################################################################################################################################
        
        
def module2(file_name,lr_txt,mincount,df):
    n_list = []#存初始名字
    for i in range(df.shape[0]):#range(df.shape[0])
        p=0
        name = df['name'][i]
        n_list.append(name)
    n_list

    txtpath = "./indel数据"
    dirs = os.listdir( txtpath )
    name_list = [] 
    for i in range(0,len(dirs)): #9.循环，不用顾虑个数
        A = dirs[i]
        name_list.append(A)

    for i in name_list:
        name1 = i
        
    data_dir = './indel结果'
    if not os.path.exists(data_dir):  
        os.mkdir(data_dir)

        
        
    result = open('./indel结果/indel_result.txt','w') # 3.result汇总到一个txt
    l = 0 #结果相同组的不同文件名
    f = 0 #检查使用左右序列次数，判断组数
    for i in name_list:
        name1 = i
        name1_list = []
        path1 = "./indel数据/%s"%name1
        dirs1 = os.listdir( path1 )
        for j in range(0,len(dirs1)):
            A = dirs1[j]
            name1_list.append(A)

            
        lr_file = open(lr_txt)
        x = 0
        for line in lr_file:
            if x>0:
                line = line.strip()
                segments = line.split(',')
                if file_name == segments[0]:
                    N20_left = segments[1].upper()
                    N20_right = segments[2].upper()
                    print (name1,N20_left,N20_right)
            x+=1
        lr_file.close()



        df_list = []
        for n in range(0,len(dirs1)):
            df1 = open('./indel数据/%s/%s'%(name1,name1_list[n]))
            df_list.append(df1)


        t = 0
        p = 0
        for df in df_list:
            insert = {}
            indel = {}
            nc = {}
            A,C,G,T={},{},{},{}
            ref_seq = {}
            t+=1
            i = 0
            exp_N20_left = N20_left.upper()
            exp_N20_right = N20_right.upper()
            delete_count,insert_count,equal_count = 0,0,0
            for line in df:
                if i>0:
                    line = line.strip()
                    segments = line.split('\t')
                    line_count = segments[1]

                    if (int(line_count)>mincount):#只运行了条数占总数大于等于100的
                        experiment = segments[0]
                        exp_N20_left_index = find_near_matches(exp_N20_left, experiment, max_l_dist=0)
                        if exp_N20_left_index:
                            exp_N20_right_index = find_near_matches(exp_N20_right, experiment, max_l_dist=0)
                            if exp_N20_right_index:  
                                exp_N20_left_end = exp_N20_left_index[0].end
                                exp_N20_right_start = exp_N20_right_index[0].start

                                exp_N20 = experiment[exp_N20_left_end:exp_N20_right_start]
                                exp_len = len(exp_N20.strip('-'))#去除前后-，是否为20
                                if exp_len<20:
                                    delete_count +=int(line_count)
                                elif exp_len == 20:
                                    equal_count +=int(line_count)
                                else:
                                    if exp_len>20:
                                        insert_count +=int(line_count)
                i+=1
            p+=1
            total = delete_count+insert_count+equal_count
            if total==0:# 4.加if，防止程序终止
                print('这一组中存在异常')
                result.write('name,delete_count,insert_count,total,delete_ratio,insert_ratio,sum_ratio')
                result.write('\n')
                result.write(n_list[l]+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常'+','+'存在异常')
                result.write('\n')
                result.write('\n')
            else:
                result.write('name,delete_count,insert_count,total,delete_ratio,insert_ratio,sum_ratio')
                result.write('\n')
                result.write(n_list[l]+','+str(delete_count)+','+str(insert_count)+','+str(total)+','+str(delete_count/total)+','+str(insert_count/total)+','+str(delete_count/total+insert_count/total))
                result.write('\n')
                result.write('\n')
            l+=1

        f+=1
        print(f)
        df.close()
        df1.close()
    result.close()
    #粘贴,存在异常


    resulttxt = 'indel_result.txt'
    resultcsv = 'indel_result.csv'

    path = './indel结果/{}'.format(resulttxt)
    f = open('./indel结果/{}'.format(resultcsv),'w',encoding='utf-8',newline='')

    csv_writer = csv.writer(f)
    with open(path,'r') as f1:
        lines = f1.readlines()
        i = 0
        for line in lines:
            if i==0 or i%3==1:
                data = line.split(',')
                #print(data)
                data1 = data[6][:-1]
                csv_writer.writerow([data[0],data[1],data[2],data[3],data[4],data[5],data1])
            i+=1
    f.close()
    
    
###################################################################################################################################
    
def module3(file_name,lr_txt,mincount,df):
    n_list = []#存初始名字
    for i in range(df.shape[0]):#range(df.shape[0])
        p=0
        name = df['name'][i]
        n_list.append(name)
    n_list

    txtpath = "./indel数据"
    dirs = os.listdir( txtpath )
    name_list = [] 
    for i in range(0,len(dirs)): #9.循环，不用顾虑个数
        A = dirs[i]
        name_list.append(A)

    for i in name_list:
        name1 = i



    data_dir = './N20'
    if not os.path.exists(data_dir):  
        os.mkdir(data_dir)

    for i in name_list:
        data_dir = './N20/%s'%(i)
        if not os.path.exists(data_dir):  
            os.mkdir(data_dir)

    data_dir = './total'
    if not os.path.exists(data_dir):  
        os.mkdir(data_dir)

    for i in name_list:
        data_dir = './total/%s'%(i)
        if not os.path.exists(data_dir):  
            os.mkdir(data_dir)


    f = 0 #检查使用左右序列次数，判断组数
    for i in name_list:


        name1 = i
        name1_list = []
        path1 = "./indel数据/%s"%name1
        dirs1 = os.listdir( path1 )
        for j in range(0,len(dirs1)):
            A = dirs1[j]
            name1_list.append(A)



        lr_file = open(lr_txt)
        x = 0
        for line in lr_file:
            if x>0:
                line = line.strip()
                #print (line)
                segments = line.split(',')
                #print (segments)
                if file_name == segments[0]:
                    N20_left = segments[1].upper()
                    N20_right = segments[2].upper()
                    N20zf = segments[3]
                    print (name1,N20_left,N20_right)
            x+=1
        lr_file.close()


        df_list = []
        for n in range(0,len(dirs1)):
            df1 = open('./indel数据/%s/%s'%(name1,name1_list[n]))
            df_list.append(df1)


        count_list = []
        for df in df_list:
            i=0
            total_count = 0
            for line in df:
            #     try:
                if i>0:
                    line = line.strip()#移除字符串头尾指定的字符（默认为空格或换行符）或字符序列
                    segments = line.split('\t')
                    line_count = segments[1] 
                    total_count+=int(line_count)
                i+=1
            count_list.append(total_count)
        df1.close()



        df_list = []
        for n in range(0,len(dirs1)):
            df1 = open('./indel数据/%s/%s'%(name1,name1_list[n]))
            df_list.append(df1)


        t = 0
        p = 0
        n = 0
        for df in df_list:
            eff = open('./N20/%s/%s'%(name1,name1_list[n]),'w')
            rtotal = open('./total/%s/%s'%(name1,name1_list[n]),'w')
            rtotall = 0
            insert = {}
            indel = {}
            nc = {}
            A,C,G,T={},{},{},{}
            ref_seq = {}
            t+=1
            i = 0
            exp_N20_left = N20_left.upper()
            exp_N20_right = N20_right.upper()
            delete_count,insert_count,equal_count = 0,0,0     
            for line in df:
                if i>0:
                    line = line.strip()
                    segments = line.split('\t')
                    line_count = segments[1] 

                    if (int(line_count)>mincount):#float(0),只运行了条数占总数大于等于1/1000的
                        experiment = segments[0] 

                        exp_N20_left_index = find_near_matches(exp_N20_left, experiment, max_l_dist=0)
                        if exp_N20_left_index:
                            exp_N20_right_index = find_near_matches(exp_N20_right, experiment, max_l_dist=0)
                            if exp_N20_right_index: 

                                exp_N20_left_end = exp_N20_left_index[0].end    
                                exp_N20_right_start = exp_N20_right_index[0].start

                                exp_N20 = experiment[exp_N20_left_end:exp_N20_right_start]
                                if N20zf == 'f':
                                    exp_N20 = complement_reverse(exp_N20)
                                exp_len = len(exp_N20)
                                if exp_len!=20:
                                      continue

                                #print(exp_len)
                                rtotall += int(line_count) #符合条件的记录总条数
                                eff.write(exp_N20+'\t'+line_count)
                                eff.write('\n')
                i+=1
            p+=1
            n+=1
            rtotal.write(str(rtotall))
            rtotal.close()
            eff.close()

        f+=1
        df.close()
        df1.close()


###################################################################################################################################

def module4(file_name,lr_txt,mincount,df):
    n_list = []#存初始名字
    for i in range(df.shape[0]):#range(df.shape[0])
        p=0
        name = df['name'][i]
        n_list.append(name)
    n_list

    txtpath = "./indel数据"
    dirs = os.listdir( txtpath )
    name_list = [] 
    for i in range(0,len(dirs)): #9.循环，不用顾虑个数
        A = dirs[i]
        name_list.append(A)
    for i in name_list:
        name1 = i
        
        
        
        
    data_dir = './statistics'
    if not os.path.exists(data_dir):  
        os.mkdir(data_dir)

    for i in name_list:
        data_dir = './statistics/%s'%(i)
        if not os.path.exists(data_dir):  
            os.mkdir(data_dir)

    f = 0 #检查使用左右序列次数，判断组数
    for i in name_list:


        name1 = i
        name1_list = []
        path1 = "./N20/%s"%name1
        dirs1 = os.listdir( path1 )
        for j in range(0,len(dirs1)):
            A = dirs1[j]
            name1_list.append(A)





        df_list = []
        for n in range(0,len(dirs1)):
            df1 = open('./N20/%s/%s'%(name1,name1_list[n]))
            df_list.append(df1)

        n = 0
        for df in df_list:
            a=1
            eff = open('./statistics/%s/%s'%(name1,name1_list[n]),'w')
            eff.write('Pos'+'\t'+'A'+'\t'+'C'+'\t'+'G'+'\t'+'T'+'\t'+'total')
            eff.write('\n')
            for j in range(0,20):
                #print(bas)
                A,C,G,T,total=0,0,0,0,0
                dff = open('./N20/%s/%s'%(name1,name1_list[n]))
                for l in dff:
                    l = l.strip()
                    segments = l.split('\t')
                    basline = segments[0]
                    basreads = segments[1]

                    if basline[j]=='A':
                        A+=int(basreads)
                    elif basline[j]=='C':
                        C+=int(basreads)
                    elif basline[j]=='G':
                        G+=int(basreads)
                    elif basline[j]=='T':
                        T+=int(basreads)
                    total+=int(basreads)

                eff.write(str(a)+'\t'+str(A)+'\t'+str(C)+'\t'+str(G)+'\t'+str(T)+'\t'+str(total))
                eff.write('\n')
                dff.close()
                a+=1

            n+=1
            eff.close()
        df1.close()


        
        
        

    data_dir = './effic'
    if not os.path.exists(data_dir):  
        os.mkdir(data_dir)

    for i in name_list:
        data_dir = './effic/%s'%(i)
        if not os.path.exists(data_dir):  
            os.mkdir(data_dir)

    f = 0 #检查使用左右序列次数，判断组数
    for i in name_list:


        name1 = i
        name1_list = []
        path1 = "./statistics/%s"%name1
        dirs1 = os.listdir( path1 )
        for j in range(0,len(dirs1)):
            A = dirs1[j]
            name1_list.append(A)




        df_list = []
        for n in range(0,len(dirs1)):
            df1 = open('./statistics/%s/%s'%(name1,name1_list[n]))
            df_list.append(df1)

        n = 0
        for df in df_list:
            a=0
            eff = open('./effic/%s/%s'%(name1,name1_list[n]),'w')

            total = open('./total/%s/%s'%(name1,name1_list[n]))    #大total
            for tot in total:
                bastotal = tot

            eff.write('Pos'+'\t'+'Aeff'+'\t'+'Ceff'+'\t'+'Geff'+'\t'+'Teff')
            eff.write('\n')
            for l in df:
                if a>0:
                    l = l.strip()
                    segments = l.split('\t')
                    baspos = segments[0]
                    basAeff = segments[1]
                    basCeff = segments[2]
                    basGeff = segments[3]
                    basTeff = segments[4]


                    Aeff = int(basAeff)/int(bastotal)
                    Aeff = '%.3g'% Aeff
                    Ceff = int(basCeff)/int(bastotal)
                    Ceff = '%.3g'% Ceff
                    Geff = int(basGeff)/int(bastotal)
                    Geff = '%.3g'% Geff
                    Teff = int(basTeff)/int(bastotal)
                    Teff = '%.3g'% Teff


                    eff.write(str(baspos)+'\t'+str(Aeff)+'\t'+str(Ceff)+'\t'+str(Geff)+'\t'+str(Teff))#+'\t'+str(total)
                    eff.write('\n')
                a+=1
            df.close()
            n+=1
            eff.close()
        df1.close()

        
        
    
    
    
def run_code_module():
    # 在这里编写你的代码模块运行逻辑
    # 例如：
    module1(bar_csv,com_fq)
    module2(file_name,lr_txt,mincount,df)
    module3(file_name,lr_txt,mincount,df)
    module4(file_name,lr_txt,mincount,df)
    # ...

    # 运行完成后，弹出消息框显示完成信息
    messagebox.showinfo("运行完成", "代码模块已全部运行完毕")
    
# 定义一个函数，用于停止运行
def start_running():
    global file_name
    global bar_csv
    global com_fq
    global lr_txt
    global mincount
    # 将全局变量设置为False，表示开始运行
    global stop_running
    global df
    stop_running = False

    # 获取输入的参数
    file_name = entry1.get()
    bar_csv = entry2.get()
    com_fq = entry3.get()
    lr_txt = entry4.get()
    mincount = int(entry5.get())
    # 在界面上显示正在运行
    status_label.config(text="正在运行...")

    # 创建一个线程来运行代码模块，以免阻塞界面
    t = threading.Thread(target=run_code_module)
    t.start()

# 定义一个函数，用于停止运行
def stop_running():
    # 将全局变量设置为True，表示停止运行
    global stop_running
    stop_running = True

    # 在界面上显示已停止运行
    status_label.config(text="已停止运行")

# 创建主窗口
window = tk.Tk()
window.title("代码模块运行器")

# 设置窗口的大小和位置
window.geometry("600x600")
window.resizable(False, False)

# 设置窗口的背景颜色
window.configure(background="#F0F0F0")

# 创建输入框和标签
label1 = tk.Label(window, text="处理点的名字:", bg="#F0F0F0", font=("Arial", 14))
label1.place(relx=0.1, rely=0.1)

entry1 = tk.Entry(window, font=("Arial", 14))
entry1.place(relx=0.4, rely=0.1)

label2 = tk.Label(window, text="barcode表格:", bg="#F0F0F0", font=("Arial", 14))
label2.place(relx=0.1, rely=0.2)

entry2 = tk.Entry(window, font=("Arial", 14))
entry2.place(relx=0.4, rely=0.2)

label3 = tk.Label(window, text="fq文件名:", bg="#F0F0F0", font=("Arial", 14))
label3.place(relx=0.1, rely=0.3)

entry3 = tk.Entry(window, font=("Arial", 14))
entry3.place(relx=0.4, rely=0.3)

label4 = tk.Label(window, text="上下游txt文件名:", bg="#F0F0F0", font=("Arial", 14))
label4.place(relx=0.1, rely=0.4)

entry4 = tk.Entry(window, font=("Arial", 14))
entry4.place(relx=0.4, rely=0.4)

label5 = tk.Label(window, text="删除reads数小于:", bg="#F0F0F0", font=("Arial", 14))
label5.place(relx=0.1, rely=0.5)

entry5 = tk.Entry(window, font=("Arial", 14))
entry5.place(relx=0.4, rely=0.5)

# 创建开始按钮
start_button = tk.Button(window, text="开始运行", command=start_running, bg="#4CAF50", fg="white", font=("Arial", 16))
start_button.place(relx=0.3, rely=0.6)

# 创建停止按钮
stop_button = tk.Button(window, text="停止运行", command=stop_running, bg="#F44336", fg="white", font=("Arial", 16))
stop_button.place(relx=0.5, rely=0.6)

# 创建状态标签
status_label = tk.Label(window, text="", bg="#F0F0F0", font=("Arial", 14))
status_label.place(relx=0.3, rely=0.7)

# 运行窗口的主循环
window.mainloop()

In [ ]:
def stop_running():
    global stop_running
    stop_running = True
    status_label.config(text="已停止运行")
status_label.config(text="正在运行...")

In [1]:
file_name = 'G2'
bar_csv = 'barcode.csv'
com_fq = 'Mut1-.fq'
lr_txt = 'N20上下游15bp正反.txt'
mincount = 50 # 2.中间长度
#N20zf = 'f'# z  f 正反
# N20_line = 'GTAAACAAAGCATAGACTGA'
# N20_line = N20_line.upper()